In [ ]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code

from LogicalQ.NoiseModel import (
    construct_noise_model,
    construct_noise_model_QuantinuumH1_1,
    construct_noise_model_QuantinuumH2_1,
    construct_noise_model_QuantinuumH2_2,
)

from LogicalQ.Benchmarks import (
    n_qubit_ghz_generation,
    quantum_teleportation,
    mirror_benchmarking,
    randomized_benchmarking,
    quantum_volume,
)

from LogicalQ.Experiments import (
    execute_circuits,
    circuit_scaling_experiment,
    noise_scaling_experiment,
    qec_cycle_efficiency_experiment,
    qec_cycle_noise_scaling_experiment,
)

from LogicalQ.Analysis import (
    circuit_scaling_bar3d,
    noise_model_scaling_bar,
    qec_cycle_efficiency_bar,
)

# Circuit Scaling

In [ ]:
# Construct circuit factories as circuit inputs
physical_circuit_factory = lambda n_qubits, circuit_length : mirror_benchmarking(n_qubits=n_qubits, circuit_length=circuit_length, measure=True)

def log_mirror_benchmarking(n_qubits, circuit_length):
    pqc = physical_circuit_factory(n_qubits, circuit_length)
    lqc = LogicalCircuit.from_physical_circuit(pqc, **steane_code)
    return lqc

logical_circuit_factory = log_mirror_benchmarking

# Construct noise model factories as noise model inputs
noise_model_factory = construct_noise_model_QuantinuumH1_1

## Serial

In [ ]:
phys_data_serial = circuit_scaling_experiment(
    physical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=64,
    shots=128,
    with_mp=False
)

In [ ]:
log_data_serial = circuit_scaling_experiment(
    logical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=1,
    shots=128,
    with_mp=False
)

In [ ]:
circuit_scaling_bar3d(phys_data_serial, title="Physical circuit scaling (serial)", show=True)

In [ ]:
circuit_scaling_bar3d(log_data_serial, title="Logical circuit scaling (serial)", show=True)

## With MP

In [ ]:
phys_data_mp = circuit_scaling_experiment(
    physical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=2,
    shots=128,
    with_mp=True
)

In [ ]:
log_data_mp = circuit_scaling_experiment(
    logical_circuit_factory,
    noise_model_factory,
    min_n_qubits=1,
    max_n_qubits=2,
    min_circuit_length=1,
    max_circuit_length=1,
    shots=128,
    with_mp=True
)

In [ ]:
circuit_scaling_bar3d(phys_data_mp, title="Physical circuit scaling (mp)", show=True)

In [ ]:
circuit_scaling_bar3d(log_data_mp, title="Logical circuit scaling (mp)", show=True)

# Noise Model Scaling

In [ ]:
# Sweep 3x3x3 = 27 combinations of noise values.
error_scan_keys = ["amplitude_damping_error_1q", "depolarizing_error_2q", "readout_error_0|1"]
error_scan_val_lists = [
    [0.0, 2e-4, 1e-3],   # 1q amplitude
    [0.0, 5e-3, 2e-2],   # 2q depol
    [0.0, 5e-3, 5e-2],   # readout 0->1
]

qc = mirror_benchmarking(n_qubits=1, circuit_length=4)
lqc = LogicalCircuit.from_physical_circuit(qc, **steane_code)

qc.measure_all()
lqc.measure_all()

base_noise_model = construct_noise_model_QuantinuumH1_1()

## Logical circuit

In [ ]:
noise_data_logical_all_true = noise_scaling_experiment(
    circuit_input=lqc,
    noise_model_input=base_noise_model,
    error_scan_keys=error_scan_keys,
    error_scan_val_lists=error_scan_val_lists,
    compute_exact=True,
    shots=1024,
    with_mp=True
)

In [ ]:
noise_data_logical_all_false = noise_scaling_experiment(
    circuit_input=lqc,
    noise_model_input=base_noise_model,
    error_scan_keys=error_scan_keys,
    error_scan_val_lists=error_scan_val_lists,
    compute_exact=False,
    shots=1,
    with_mp=False
)

In [ ]:
noise_model_scaling_bar(noise_data_logical_all_true, scan_keys=error_scan_keys, separate_plots=True)

In [ ]:
noise_model_scaling_bar(noise_data_logical_all_false, scan_keys=error_scan_keys, separate_plots=False)

## Physical circuit

In [ ]:
noise_data_phys_all_true = noise_scaling_experiment(
    circuit_input=qc,
    noise_model_input=base_noise_model,
    error_scan_keys=error_scan_keys,
    error_scan_val_lists=error_scan_val_lists,
    compute_exact=True,
    shots=1024,
    with_mp=True
)

In [ ]:
noise_data_phys_all_false = noise_scaling_experiment(
    circuit_input=qc,
    noise_model_input=base_noise_model,
    error_scan_keys=error_scan_keys,
    error_scan_val_lists=error_scan_val_lists,
    compute_exact=True,
    shots=256,
    with_mp=False
)

In [ ]:
noise_model_scaling_bar(noise_data_phys_all_true, scan_keys=error_scan_keys, separate_plots=True)

In [ ]:
noise_model_scaling_bar(noise_data_phys_all_false, scan_keys=error_scan_keys, separate_plots=False)

# QEC Cycle Efficiency

In [ ]:
# Insert varying number of QEC cycles and see how fidelity behaves
constraint_scan_keys = ["effective_threshold", "n_gates"]
constraint_scan_val_lists = [[1e-3, 2e-3, 5e-3], list(range(5))]

qc = mirror_benchmarking(n_qubits=1, circuit_length=4)

# Concrete noise model (base Quantinuum H1-1)
base_noise_model = construct_noise_model_QuantinuumH1_1()

## Serial

In [ ]:
qec_eff_data_serial = qec_cycle_efficiency_experiment(
    circuit_input=qc,
    noise_model_input=base_noise_model,
    qecc=steane_code,
    constraint_scan_keys=constraint_scan_keys,
    constraint_scan_val_lists=constraint_scan_val_lists,
    method="density_matrix",
    shots=256,
    with_mp=False
)

In [ ]:
qec_cycle_efficiency_bar(qec_eff_data_serial)

## With MP

In [ ]:
qec_eff_data_mp = qec_cycle_efficiency_experiment(
    circuit_input=qc,
    noise_model_input=base_noise_model,
    qecc=steane_code,
    constraint_scan_keys=constraint_scan_keys,
    constraint_scan_val_lists=constraint_scan_val_lists,
    method="density_matrix",
    shots=256,
    with_mp=True
)

In [ ]:
qec_cycle_efficiency_bar(qec_eff_data_mp)

# QEC Cycle Noise Scaling

In [ ]:
qec_noise_keys = ["p1q"]
qec_noise_vals = [[0.0, 1e-2]]

In [ ]:
qec_noise_scaling_serial = qec_cycle_noise_scaling_experiment(
    circuit_input=qc,
    noise_model_input=base_noise_model,
    qecc=steane_code,
    constraint_scan_keys=constraint_scan_keys,
    constraint_scan_val_lists=constraint_scan_val_lists,
    error_scan_keys=qec_noise_keys,
    error_scan_val_lists=qec_noise_vals,
    save_filename=None,
    with_mp=False
)

In [ ]:
qec_noise_scaling_mp = qec_cycle_noise_scaling_experiment(
    circuit_input=qc,
    noise_model_input=base_noise_model,
    qecc=steane_code,
    constraint_scan_keys=constraint_scan_keys,
    constraint_scan_val_lists=constraint_scan_val_lists,
    error_scan_keys=qec_noise_keys,
    error_scan_val_lists=qec_noise_vals,
    save_filename=None,
    with_mp=True
)